<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/dataset/create_1%2C2%2C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai ragas datasets faiss-cpu groq langchain_community evaluate anls sentence_transformers
!pip install -U langchain-openai
import json
from itertools import combinations
import openai
from google.colab import drive, userdata
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY2')

## 문장 생성

In [ ]:
# LLM 호출 함수
def generate_sentence(combination):
    # 키워드 C를 가져오기 (모든 조합에서 동일함)
    C_keyword = combination[0][2]

    # 입력 데이터를 문장으로 변환
    sentences = []
    for entry in combination:
        if len(entry) >= 4:
            A, B, C, D = entry  # C는 제외
            sentences.append(f"{A} {D} is {B}.")
        else:
            # Handle cases where entry has fewer values, e.g., print a warning
            print(f"Warning: Entry has fewer than 4 values: {entry}")


    # C 키워드를 한 번만 포함한 메시지 생성
    prompt = f"The following information is about {C_keyword}: " + " ".join(sentences)

    # LLM에게 보낼 메시지
    messages = [
        {"role": "system", "content": "Generate a natural English sentence combining the following information without repeating the keyword."},
        {"role": "user", "content": prompt}
    ]

    # OpenAI ChatGPT API 호출
    response = openai.chat.completions.create(
        model="gpt-4o",  # 모델 선택
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content.strip()

# 입력 파일 경로
input_file = "/content/data.txt"

# 출력 파일 경로
output_file = "/content/output.json"

# 조합 크기 설정 (2, 3, 4 등으로 변경 가능)
combination_size = 1

# 데이터 읽기
with open(input_file, "r") as file:
    data = file.readlines()

# 데이터 파싱 및 그룹화
parsed_data = []
for line in data:
    # 데이터 파싱: 공백 제거 및 정확히 4개로 분리
    entry = [item.strip() for item in line.strip().split(" | ")]
    if len(entry) == 4:  # 정확히 4개의 요소만 추가
        parsed_data.append(entry)
    else:
        print(f"잘못된 형식 무시: {line.strip()}")

# 그룹화
grouped_data = {}
for entry in parsed_data:
    A, B, C, D = entry
    grouped_data.setdefault(C, []).append(entry)

In [ ]:
# 조합 생성 및 문장 생성
results = []
for C, entries in grouped_data.items():
    if len(entries) < combination_size:
        # 조합할 데이터가 부족하면 건너뛰기
        print(f"그룹 '{C}'는 조합 크기 {combination_size}개를 충족하지 않아 건너뜁니다.")
        continue

    # 지정된 크기만큼 조합 생성
    combinations_list = list(combinations(entries, combination_size))
    for combination in combinations_list:
        sentence = generate_sentence(combination)
        results.append({
            "combination": combination,
            "sentence": sentence
        })

# 결과 저장
with open(output_file, "w") as file:
    json.dump(results, file, indent=4, ensure_ascii=False)

print(f"결과가 {output_file}에 저장되었습니다.")

결과가 /content/output.json에 저장되었습니다.


## 복합 질문 생성

In [ ]:
import re

In [ ]:
# LLM 호출 함수: 질문과 간단한 키워드 형태의 정답 생성
def generate_question_and_answer(sentence1, sentence2):
    prompt = (
        f"Combine the following sentences into one meaningful question, ensuring that the same keyword does not appear twice. "
        f"Make sure to include all information from both sentences without omitting any details:\n\n"
        f"Sentence 1: {sentence1}\n"
        f"Sentence 2: {sentence2}\n\n"
        f"Then, provide the simplest possible answer containing only key facts (e.g., numerical values, dates, or specific terms) that directly address the question.\n"
        f"Format your response as follows:\n"
        f"Question: <Your question here>\nAnswer: <Key facts or keywords here>"
    )
    messages = [
        {"role": "system", "content": "You are an assistant specializing in creating concise questions and extracting key facts as answers."},
        {"role": "user", "content": prompt}
    ]
    # OpenAI ChatGPT API 호출
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    # 결과 파싱
    response_content = response.choices[0].message.content.strip()
    question, *answer = response_content.split("\nAnswer:")
    question = question.replace("Question: ", "").strip()  # "Question:" 제거
    answer = answer[0].strip() if answer else "Could not generate answer"
    return question, answer


In [ ]:
# JSON 파일 경로 설정
input_folder = "/content/"
sentence_files = ["sentence1.json", "sentence2.json", "sentence3.json", "sentence4.json"]

# JSON 파일 로드
data = {}
for file_name in sentence_files:
    file_path = os.path.join(input_folder, file_name)
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data[file_name] = json.load(file)
    else:
        print(f"{file_name} 파일이 존재하지 않습니다. 무시합니다.")

# 조합 생성 (sentence1에서 1개, sentence4에서 1개씩 조합)
results = []
for sentence1_data in data.get("sentence1.json", []):
    sentence1 = sentence1_data["sentence"]
    keyword1 = sentence1_data["combination"][0][2]  # sentence1의 키워드(C)

    for sentence4_data in data.get("sentence4.json", []):
        sentence4 = sentence4_data["sentence"]
        keyword4 = sentence4_data["combination"][0][2]  # sentence4의 키워드(C)

        # 키워드가 동일한 경우 건너뛰기
        if keyword1 == keyword4:
            continue

        # 질문 및 간단한 키워드 형태의 답변 생성
        question, answer = generate_question_and_answer(sentence1, sentence4)

        results.append({
            "sentence4": sentence4,
            "sentence1": sentence1,
            "question": question,
            "answer": answer
        })

# 결과 저장
output_file = "/content/QA_4_1.json"
with open(output_file, "w") as file:
    json.dump(results, file, indent=4, ensure_ascii=False)

print(f"질문 및 간단한 정답 생성 결과가 {output_file}에 저장되었습니다.")

KeyboardInterrupt: 

## 괄호 없애기

In [ ]:
import json

# JSON 파일 경로
input_file_path = '/content/sentence4.json'
output_file_path = '/content/sentence4.json'

# JSON 파일 로드
with open(input_file_path, 'r') as file:
    data = json.load(file)

# combination 필드에서 첫 번째와 마지막 부분을 제거
for item in data:
    for combination in item.get('combination', []):
        # 첫 번째 요소와 마지막 요소를 제거
        combination[0] = combination[0][1:]  # 첫 번째 '('를 제거
        combination[-1] = combination[-1][:-1]  # 마지막 ')'를 제거

# 변경된 데이터를 새 파일로 저장
with open(output_file_path, 'w') as file:
    json.dump(data, file, indent=4)

print("변경이 완료되었습니다.")


변경이 완료되었습니다.


## 질문 하드코딩_V1 (D를 이용)

In [ ]:
import json

# JSON 파일 경로
input_file_path = '/content/sentence4.json'
txt_file_path = '/content/data.txt'
output_file_path = '/content/output_file4.json'

# JSON 파일 로드
with open(input_file_path, 'r') as file:
    data = json.load(file)

# txt 파일 로드
with open(txt_file_path, 'r') as file:
    txt_lines = file.readlines()

# txt 파일을 (A | B | C | D) 형태로 처리
txt_data = []
for line in txt_lines:
    parts = line.strip().split(' | ')
    if len(parts) == 4:
        # D 부분에서 ")" 제거
        parts[3] = parts[3].rstrip(')')
        txt_data.append(parts)

# combination의 3번째와 txt의 C를 비교하고 D를 추가
for item in data:
    item['question'] = []  # Initialize the question list
    item['answer'] = []  # Initialize the answer list
    C_value_set = set()  # Set to store unique C_value from the combination

    # Step 1: Collect all D values for each combination
    D_values = {}  # To store D values for each C_value
    for combination in item.get('combination', []):
        C_value = combination[2]  # combination의 3번째 값 (C)
        D_value = combination[3]  # combination의 4번째 값 (D)

        if C_value not in D_values:
            D_values[C_value] = set()
        D_values[C_value].add(D_value)

    # Step 2: Check txt file for matching C and D values
    for C_value, D_value_set in D_values.items():
        for txt in txt_data:
            if txt[2] == C_value:
                # Step 3: For each C_value, check D values and create questions for missing D values
                for D_value in D_value_set:
                    if txt[3] != D_value:  # D 값이 txt에서 존재하지 않으면
                        # txt[3]이 D_values에 이미 있으면 질문을 만들지 않음
                        if txt[3] in D_value_set:
                            continue

                        question = f"what is {txt[3]} of {item['sentence'].rstrip('.') }?"
                        # 문장에서 마지막 마침표 제거 후 물음표 추가

                        # question을 리스트에 추가
                        if question not in item['question']:
                            item['question'].append(question)
                            # B 값을 answer로 추가
                            answer = txt[1]  # B 값은 answer 필드에 저장
                            item['answer'].append(answer)  # answer를 리스트에 추가



# 변경된 데이터를 새 파일로 저장
with open(output_file_path, 'w') as file:
    json.dump(data, file, indent=4)

print("변경이 완료되었습니다.")


변경이 완료되었습니다.


## 질문 하드코딩_V2 (A이용)

## 질문 잇기

In [ ]:
import json

# JSON 파일 경로
outputfile1_path = '/content/output_file4.json'
outputfile2_path = '/content/output_file1.json'
outputfile3_path = '/content/QA_4_1.json'

# outputfile1.json과 outputfile2.json 로드
with open(outputfile1_path, 'r') as file:
    data1 = json.load(file)

with open(outputfile2_path, 'r') as file:
    data2 = json.load(file)

# 새로운 리스트를 저장할 변수
new_data = []

# Step 1: 질문 결합 (겹치지 않는 C 값을 기반으로)
for item1 in data1:
    for item2 in data2:
        # C 값 비교 (combination의 첫번째 항목에서 3번째 값)
        C_value_1 = item1['combination'][0][2]
        C_value_2 = item2['combination'][0][2]

        # C 값이 겹치지 않으면
        if C_value_1 != C_value_2:
            # 기존 질문을 이어 붙이기 (물음표를 제거하고 'and'로 연결)
            for question1, answer1 in zip(item1['question'], item1['answer']):
                for question2, answer2 in zip(item2['question'], item2['answer']):
                    # 물음표 제거하고 'and' 추가
                    new_question = question1.rstrip('?') + " and " + question2.rstrip('?') + "?"

                    # 새로운 질문과 답변을 리스트에 추가
                    new_data.append({
                        'question': new_question,
                        'gt_ans': [answer1, answer2]  # 답변을 리스트 형식으로 저장
                    })

# 새로운 JSON 파일로 저장
with open(outputfile3_path, 'w') as file:
    json.dump(new_data, file, indent=4)

print("새로운 JSON 파일이 생성되었습니다.")


새로운 JSON 파일이 생성되었습니다.
